<a href="https://colab.research.google.com/github/antoniomuso/speech2face/blob/master/Speech2Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# path = "/content/drive/My Drive/Speech2Face/vox"
# !curl --user voxceleb1912:0s42xuw6 -o "/content/drive/My Drive/Speech2Face/ff/vox.zip" http://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox1a/vox1_test_wav.zip

In [ ]:
# !unzip "/content/drive/My Drive/Speech2Face/ff/vox.zip" -d "/content/drive/My Drive/Speech2Face/ff/ext/"

In [ ]:
import librosa
import numpy as np

In [ ]:
wav, sr = librosa.load('/content/drive/My Drive/Speech2Face/vox1_dataset/vox_audios/ext/wav/id10270/5sJomL_D0_g/00001.wav',sr = 16000, duration = 6.0 ,mono = True) 

In [ ]:
spectro = librosa.core.stft(wav, n_fft = 512, hop_length = int(np.ceil(0.01 * sr)),win_length = int(np.ceil(0.025 * sr)) , window='hann', center=True,pad_mode='reflect')

In [ ]:
def adjust(stft):
  if stft.shape[1] == 601:
    return stft
  else:
    return np.concatenate((stft,stft[:,0:601 - stft.shape[1]]),axis = 1)

spectroComplex = adjust(spectro)
converted = np.zeros((spectroComplex.shape[0], spectroComplex.shape[1], 2))
i = np.arange(spectroComplex.shape[0])
j = np.arange(spectroComplex.shape[1])

converted[i,j[:,np.newaxis], 0] = spectroComplex[i,j[:,np.newaxis]].real
converted[i,j[:,np.newaxis], 1] = spectroComplex[i,j[:,np.newaxis]].imag

In [ ]:
import torch
import torch.nn as nn

class SpeechEncoder(nn.Module):

    def __init__(self):
        super(SpeechEncoder, self).__init__()
        self.conv1 = nn.Conv2d(2, 64, kernel_size=4,stride=1) 
        self.conv2 = nn.Conv2d(64, 64, kernel_size=4,stride=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4,stride=1) 
        self.pooling1 = nn.MaxPool2d(kernel_size=(2,1), stride=(2,1))
        self.conv4 = nn.Conv2d(128, 128, kernel_size=4,stride=1) 
        self.pooling2 = nn.MaxPool2d(kernel_size=(2,1), stride=(2,1))
        self.conv5 = nn.Conv2d(128, 128, kernel_size=4,stride=1) 
        self.pooling3 = nn.MaxPool2d(kernel_size=(2,1), stride=(2,1))
        self.conv6 = nn.Conv2d(128, 256, kernel_size=4,stride=1) 
        self.pooling4 = nn.MaxPool2d(kernel_size=(2,1), stride=(2,1))
        self.conv7 = nn.Conv2d(256, 512, kernel_size=4,stride=1) 
        self.conv8 = nn.Conv2d(512, 512, kernel_size=4,stride=2) 

        self.conv9 = nn.Conv2d(512, 512, kernel_size=3,stride=2) # Queste due celle sono diverse
        self.pooling5 = nn.AvgPool2d(kernel_size=(1,1), stride=1)# Queste due celle sono diverse

        self.fc1 = nn.Linear(512 * 1 * 144, 4096)
        self.fc2 = nn.Linear(4096, 4096)

        self.batch_norm1 = nn.BatchNorm2d(64)
        self.batch_norm2 = nn.BatchNorm2d(64)
        self.batch_norm3 = nn.BatchNorm2d(128)
        self.batch_norm4 = nn.BatchNorm2d(128)
        self.batch_norm5 = nn.BatchNorm2d(128)
        self.batch_norm6 = nn.BatchNorm2d(256)
        self.batch_norm7 = nn.BatchNorm2d(512)
        self.batch_norm8 = nn.BatchNorm2d(512)
        self.batch_norm9 = nn.BatchNorm2d(512)
      


        self.relu = nn.ReLU()
        

    def forward(self, x):
        out = self.batch_norm1(self.relu(self.conv1(x)))
        out = self.batch_norm2(self.relu(self.conv2(out)))
        out = self.batch_norm3(self.relu(self.conv3(out)))
        out = self.pooling1(out)
        out = self.batch_norm4(self.relu(self.conv4(out)))
        out = self.pooling2(out)
        out = self.batch_norm5(self.relu(self.conv5(out)))
        out = self.pooling3(out)
        out = self.batch_norm6(self.relu(self.conv6(out)))
        out = self.pooling4(out)
        out = self.batch_norm7(self.relu(self.conv7(out)))
        out = self.batch_norm8(self.relu(self.conv8(out)))
        out = self.batch_norm9(self.relu(self.pooling5(self.conv9(out))))

        batch = out.shape[0]
        out = out.view((batch, 512 * 1 * 144))
        out = self.relu(self.fc1(out))
        out = self.fc2(out)

        return out



In [ ]:
model = SpeechEncoder()
input = torch.unsqueeze(torch.tensor(converted).reshape(2,257,601), 0)

model(input.type(torch.float32)).shape